# Sports Features

This notebook reads Red Sox, Celtics, and Bruins home game information and engineers features from them.

## Setup

In [98]:
# Libraries.
import datetime as dt
import numpy as np
import pandas as pd
import re

In [77]:
# Library settings.

# Pandas.
pd.set_option('display.max_columns', 1000)

In [78]:
# Other miscellanea.

# Abbreviations for teams.
teams = ['bruins','celtics','sox']

# Approximate game lengths based on a cursory web search of typical MLB/NBA/NHL lengths.
game_lengths = {
    'bruins':  2.3333,
    'celtics': 2.25,
    'sox': 3+8./60.
}

# Weekday abbreviations.
weekday_abbreviations = 'mon,tue,wed,thu,fri,sat,sun'.split(',')

# Cutoff for early/late game times.
late_game_start = dt.time(18)

## Data

In [79]:
# Read Bruins home games.
bruins = pd.DataFrame.from_csv("../../../data/bruins/home.csv").reset_index(drop=False)

# Drop home column.
bruins.drop(['Home'], axis=1, inplace=True)

# Rename other columns to make joining easy.
bruins.rename(columns={'Datetime':'bruins_datetime', 'Opponent':'bruins_opponent', 'Playoff': 'bruins_playoff'}, inplace=True)

# Get standalone game date, time.
bruins['date'] = pd.DatetimeIndex(bruins.bruins_datetime).date
bruins['bruins_time'] = pd.DatetimeIndex(bruins.bruins_datetime).time

# Drop datetime.
bruins.drop(['bruins_datetime'], axis=1, inplace=True)

In [ ]:
bruins.head()

In [81]:
# Read Celtics home games.
celtics = pd.DataFrame.from_csv("../../../data/celtics/home.csv").reset_index(drop=False)

# Drop home/playoff columns.
celtics.drop(['Home'], axis=1, inplace=True)

# Rename other columns to make joining easy.
celtics.rename(columns={'Datetime':'celtics_datetime', 'Opponent':'celtics_opponent', 'Playoff': 'celtics_playoff'}, inplace=True)

# Get standalone game date, time.
celtics['date'] = pd.DatetimeIndex(celtics.celtics_datetime).date
celtics['celtics_time'] = pd.DatetimeIndex(celtics.celtics_datetime).time

# Drop datetime.
celtics.drop(['celtics_datetime'], axis=1, inplace=True)

In [ ]:
celtics.head()

In [83]:
# Read Red Sox home games.
sox = pd.DataFrame.from_csv("../../../data/sox_master.csv").reset_index(drop=True)

# Rename other columns to make joining easy.
sox.rename(columns={'START_DATE':'date', 'START_TIME':'sox_time', 'OPPONENT':'sox_opponent'}, inplace=True)

# Drop 0s from start date.
sox.date = sox.date.str.extract('(\d+-\d+-\d+)')
# Column for start datetime.
sox['sox_datetime'] = pd.to_datetime(sox.date + ' ' + sox.sox_time)

# Add a "rounded" game time to make grouping easier.
sox['sox_time_rounded'] = sox.sox_time.str.replace(':[012][05] ',':00 ').str.replace(':[345][05] ',':30 ')
sox['sox_datetime_rounded'] = pd.to_datetime(sox.date + ' ' + sox.sox_time_rounded)

# Turn start date into a date.
sox.date = pd.DatetimeIndex(sox.date).date

# Convert time fields to times.
sox.sox_time = pd.DatetimeIndex(sox.sox_datetime).time
sox.sox_time_rounded = pd.DatetimeIndex(sox.sox_datetime_rounded).time

# Drop datetime.
sox.drop(['sox_datetime','sox_datetime_rounded'], axis=1, inplace=True)

In [ ]:
sox.head()

## Merge

In [85]:
# Get game dates.
game_dates = sorted(list(set(bruins.date.append(celtics.date).append(sox.date))))

# Start DF.
games = pd.DataFrame({'date':game_dates})

# Basic weekday column.
games['weekday'] = pd.DatetimeIndex(games.date).weekday

# Merge to each team.
games = (
    games.merge(bruins,  on='date', how='left')
         .merge(celtics, on='date', how='left')
         .merge(sox,     on='date', how='left')
)

# Pre-feature-engineering column count.
pre_feature_engineering_d = games.shape[1]

In [ ]:
games.head()

## Features

Derive binary features here. Keep as booleans for now; will turn into 0/1 ints later.

In [87]:
# Simple team game features.
for team in teams:
    games[team+'_game'] = games[team+'_opponent'].isnull() == False

In [88]:
# Day of week features.
# These include one-hot weekday columns and special groupings: M-Th, M-F, F-Sa.
for team in teams:
    for d in xrange(7):
        games[team+'_'+weekday_abbreviations[d]] = games[team+'_game'] & (games.weekday == d)
    games[team+'_mon_thu'] = games[team+'_game'] & (games.weekday <= 3)
    games[team+'_mon_fri'] = games[team+'_game'] & (games.weekday <= 4)
    games[team+'_fri_sat'] = games[team+'_game'] & (games.weekday.isin([4,5]))

In [89]:
# Early/late features.
for team in teams:
    games[team+'_time_early'] = games[team+'_time'] <  late_game_start
    games[team+'_time_late']  = games[team+'_time'] >= late_game_start

In [90]:
# Game day-time intersection features.

# Suffixes for game day group features.
game_day_group_suffixes  = weekday_abbreviations+['mon_thu', 'mon_fri', 'fri_sat']
game_time_group_suffixes = ['early','late']

# Container for useful game feature columns.
game_cols = []

# One set per team.
for team in teams:
    day_group_cols  = [team+'_'+s for s in game_day_group_suffixes]
    time_group_cols = [team+'_game_'+s for s in game_time_group_suffixes]
    for day_suffix in game_day_group_suffixes:
        for time_suffix in game_time_group_suffixes:
            # Calculate each column.
            col_name = team+'_'+day_suffix+'_'+time_suffix
            games[col_name] = games[team+'_'+day_suffix] & games[team+'_time_'+time_suffix]
            # Save these columns too. We're interseted in using them as final features.
            game_cols.append(col_name)

In [91]:
# Prune game columns to remove overlaps.
# game_cols = [col for col in game_cols if ('_mon_fri_' not in col and '_fri_sat_' not in col)]

All features are derived at this point. Make 0/1 binary.

In [92]:
for col in games.columns[pre_feature_engineering_d:]:
    games[col] = games[col].astype(np.int)

In [ ]:
games

## Export

In [94]:
games.to_csv('../../../data/game_days.csv', index=False)

## Exploration

Verify viability of specific features.

### Bruins

In [164]:
## Specific columns we're interested in.
bruins_game_cols = [col for col in games.columns if re.match(r'bruins_(mon_fri_early|mon_thu_late|fri_sat_late|fri_sat_late|sat_early|sun$)', col)]

# Get features.
bruins_features = games.ix[games.bruins_game == 1,bruins_game_cols]

# Record counts.
bruins_feature_counts = pd.DataFrame(bruins_features.groupby(bruins_features.columns.tolist()).size())
bruins_feature_counts.columns = ['count']
bruins_feature_counts

count
bruins_sun bruins_sat_early bruins_mon_thu_late bruins_mon_fri_early bruins_fri_sat_late       
0          0                0                   0                    1                       22
                                                1                    0                        6
                            1                   0                    0                       69
           1                0                   0                    0                       21
1          0                0                   0                    0                        9

### Celtics

In [152]:
# Specific columns we're interested in.
celtics_game_cols = [col for col in games.columns if re.match(r'celtics_(mon_thu|fri_sat|sun)$', col)]

# Get features.
celtics_features = games.ix[games.celtics_game == 1,celtics_game_cols]

# Record counts.
celtics_feature_counts = pd.DataFrame(celtics_features.groupby(celtics_features.columns.tolist()).size())
celtics_feature_counts.columns = ['count']
celtics_feature_counts

count
celtics_sun celtics_mon_thu celtics_fri_sat       
0           0               1                   47
            1               0                   58
1           0               0                   17

### Red Sox

In [157]:
# Specific columns we're interested in.
sox_game_cols = [col for col in game_cols if re.match(r'sox_(mon_(fri_early|thu_late)|fri_late|((sat|sun)_(early|late)))', col)]

# Get features.
sox_features = games.ix[(games.sox_game == 1) & (games.date >= dt.date(2013,1,1)),sox_game_cols]

# Record counts.
sox_feature_counts = pd.DataFrame(sox_features.groupby(sox_features.columns.tolist()).size())
sox_feature_counts.columns = ['count']
sox_feature_counts

count
sox_fri_late sox_sat_early sox_sat_late sox_sun_early sox_sun_late sox_mon_thu_late sox_mon_fri_early       
0            0             0            0             0            0                1                     16
                                                                   1                0                     74
                                                      1            0                0                      8
                                        1             0            0                0                     20
                           1            0             0            0                0                     14
             1             0            0             0            0                0                     17
1            0             0            0             0            0                0                     21